Загрузить данные из таблицы test.xlsx.  
Проанализировать текст в колонке text, используя метод обработки текста.  
Сгенерировать вероятный краткий ответ из всего текста (состоящий из нескольких слов) и записать его в колонку answer.  

Пример:  
Текст1 - Удовлетворить  
Текст2 - Производство окончено  
Текст3 - Отказ, ИП приостановлено  
Текст4 - Отказ, невозможно установить местонахождение должника  

Такую задачу можно просто сделать через LLM, через OpenRouter с бесплатными моделями, но тут будет хромать скорость и лимиты запросов, доступность сервиса. Если же сделать через платный API, то нужны постоянные дополнительные затраты.
Так что можно сделать через более простые способы, попробовать линейные модели и бустинг модели, базовые обработки текста и так далее.

## Обработка данных

In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [2]:
df = pd.read_excel('data/orig/test.xlsx')
df.head()

,Id,text,answer
0,672527f94ac0e6c927fe9209,"Обратить\t взыскание\t на\t денежные\t средства\t должника\t КОВАЛЕВ\t ВЯЧЕСЛАВ\t ГЕННАДЬЕВИЧ\t 03.09.1978\nг.р.,\tнаходящиеся\tна\tсчетах\tв\tбанках\tи\tиных\tкредитных\tорганизациях:\nЮГО-ЗАПАДНЫЙ\tБАНК\tПАО\tСБЕРБАНК,\tБИК\t046015602,\tсчет:\t40817810630061819252;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601398000003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601398500003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601933800003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601933300003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840100003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840600003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810800003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810700001700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978200003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810300003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840000001700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978700003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840500001700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810200001700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978600001700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978100001700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601156200003700124;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t30601810205052519251;\nФИЛИАЛ\tNo\t2351\tБАНКА\tВТБ\t(ПАО),\tБИК\t040349758,\tсчет:\t40817840932556001971;\nФИЛИАЛ\tNo\t2351\tБАНКА\tВТБ\t(ПАО),\tБИК\t040349758,\tсчет:\t40817810832556010898;\nЮГО-ЗАПАДНЫЙ\tБАНК\tПАО\tСБЕРБАНК,\tБИК\t046015602,\tсчет:\t40817810530062328130;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t40817810722254002807;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t40817978222254002168;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t40817840622254002168;\nКБ\t""Ренессанс\tКредит""\t(ООО),\tБИК\t044525135,\tсчет:\t40817810997750205910;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810938049607994;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810038064928749;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810038064927313;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810138170573768;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810540005904391;\nПАО\tРОСБАНК,\tБИК\t044525256,\tсчет:\t40817810097440007054;\nКУРСКОЕ\tОТДЕЛЕНИЕ\tN8596\tПАО\tСБЕРБАНК,\tБИК\t043807606,\tсчет:\t42307810033000116032;\nАО\t""АЛЬФА-БАНК"",\tБИК\t044525593,\tсчет:\t40817810506370028148;\nБАНК\tГПБ\t(АО),\tБИК\t044525823,\tсчет:\t40817810964071606339;\nБАНК\tГПБ\t(АО),\tБИК\t044525823,\tсчет:\t40817810364071611665;\nФИЛИАЛ\tЦЕНТРАЛЬНЫЙ\tПАО\tБАНКА\t""ФК\tОТКРЫТИЕ"",\tБИК\t044525297,\tсчет:\t40817810978006973068;\nФИЛИАЛ\tNo\t2351\tБАНКА\tВТБ\t(ПАО),\tБИК\t040349758,\tсчет:\t40817978432556001929;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601156700003700129;\nнезависимо\tот\tостатка\tденежных\tсредств\tна\tсчете.",NaN
1,672526134ac0e6c927fe3400,"Обратить\t взыскание\t на\t денежные\t средства\t должника\t Арбузова\t Михаила\t Михайловича\t 03.03.1976\t г.р.,\nнаходящиеся\tна\tсчетах\tв\tбанках\tи\tиных\tкредитных\tорганизациях:\nАО\t""ТИНЬКОФФ\tБАНК"",\tБИК\t044525974,\tсчет:\t40817810600001600017;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307810046140126000;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307810646007402351;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307810346005600057;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t40817810146007400345;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307840146007400030;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК

In [3]:
# Проверяем на пустые значения в столбцах, если вдруг надо обработать их
df.isnull().sum()

Id           0
text         0
answer    1000
dtype: int64

Так как цифры встречаются очень часто и не несут смысловой нагрузки, можем обработать их, как и знаки препинания

In [4]:
def apply_and_check(dataframe, func, column, target_column=None, verbose=True, **kwargs):
    if target_column is None:
        target_column = column

    total_before = dataframe[column].str.len().sum()
    dataframe[target_column] = dataframe[column].apply(func, **kwargs)

    total_after = dataframe[target_column].str.len().sum()
    total_removed = total_before - total_after
    
    if verbose:
        print(f"Символов до: {total_before:,}, "
              f"после: {total_after:,}")
        print(f"Удалено символов: {total_removed:,} ({total_removed/total_before*100:.2f}%)")

### Символы

In [5]:
import re

# Удаляем табуляции/переносы строк, цифры, знаки препинания и приводим к нижнему регистру
def clean_text(text):
    # text = re.sub(r"[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-\t\n№]+", ' ', text).lower()
    
    text = re.sub(r'[^а-яё\s]+', ' ', text.lower()) # Оставляем только кириллицу
    text = re.sub(r'\s+', ' ', text).strip()  # Убираем табуляцию/переносы и склеиваем все whitespace в один пробел
    return text

In [6]:
apply_and_check(df, clean_text, 'text', 'clean')

Символов до: 1,036,674, после: 893,443
Удалено символов: 143,231 (13.82%)


### Стоп-слова

In [7]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words("russian")

# Оставляем только "не" из всех отдельных стоп-слов для последующего соеденения после лемматизации
stopwords_ru.remove('не')

def remove_stopwords(text):
    # Если попадётся не слово или нуль
    if not isinstance(text, str) or not text.strip():
        return text
    
    tokens = text.split()
    
    filtered_tokens = [token for token in tokens 
                       if token and token not in stopwords_ru]
    
    return ' '.join(filtered_tokens)

In [8]:
apply_and_check(df, remove_stopwords, 'clean')

Символов до: 893,443, после: 835,537
Удалено символов: 57,906 (6.48%)


### Лишние слова по частоте

Также нужно убрать часто встречающиеся лишние слова по типу "счёт, банк" и подобные  
Выберем из часто встречающихся слов через value_counts(), которые могут не нести смысла для классификации

In [9]:
# Подсчет слов
word_counts = df['clean'].str.split().explode().value_counts()

# Вывод топ слов
word_counts.head(50)

clean
не                 3150
указано            1892
сведений           1661
банк               1563
запрос             1486
дата               1252
мвв                1074
должника            973
счета               770
россии              735
г                   713
исполнительного     661
актуальности        647
сумме               641
пао                 596
сведения            584
постановление       579
производства        567
счет                551
средств             545
сумма               518
связи               508
денежных            493
бик                 473
код                 468
удовлетворению      468
ип                  466
ао                  463
признает            452
валюта              450
заявителя           445
наличие             405
обстоятельства      400
заявлении           389
судебного           386
подразделения       383
рассмотрев          381
валюте              378
доводы              372
счете               365
единица             359
курса     

In [10]:
# Выберем те, которые кажутся лишними
# Точно надо удалить их перед проверкой орфографии как можно больше, так как слишком долго обрабатывает,
# но и искать каждое слово в каждом падеже как-то неправильно. Может снова пройтись после лемматизации, так как это может убрать ещё слова
unnecessary_words = {
    "счет", "счета", "банк", "банка", "мвв", "г", "валюта", "сумма",
    "сумме", "рубль", "курс", "бик", "код", "адрес", "россия", "россии", 
    "филиал", "отделение"
}

def remove_unnecessary_words(text):
    pattern = r'\b(' + '|'.join(re.escape(word) for word in unnecessary_words) + r')\b'
    return re.sub(pattern, '', text, flags=re.IGNORECASE)

In [11]:
apply_and_check(df, remove_unnecessary_words, 'clean')

Символов до: 835,537, после: 796,391
Удалено символов: 39,146 (4.69%)


В итоге после базовой очистки осталось ~80% символов всего датасета

### Отдельный столбец для методов выделения ключевых слов

Для этих методов нужно убрать цифры, но не пунктуацию и стоп-слова

In [12]:
# Отдельный класс под эти методы. Тут сложновато, так как даты, хар-ки машин, счета и другие создают множество пробелов и пустых пунктуаций
def clean_text_keyword(text, lower=True):
    # Приводим к нижнему регистру
    if lower:
        text = text.lower()

    # Убираем отдельно обозначения машин и подобные, так как они состоят не только из цифр
    text = re.sub(r'\bvin\s*[a-z0-9]{17}\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b(?=.*\d)(?=.*[a-z])[a-z0-9]{6,}\b', '', text, flags=re.IGNORECASE)
    
    # Убираем ВСЕ цифры
    text = re.sub(r'\d+', '', text)
    
    # Убираем повторяющиеся одинаковые знаки пунктуации (.,!? и т.д.)
    # "????" -> "?", ",...." -> ",."
    text = re.sub(r'([.,!?;:—\-])\1+', r'\1', text)
    
    # Убираем множественные пробелы (включая пробелы вокруг пунктуации)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Убираем пунктуацию, стоящую отдельно от слов
    text = re.sub(r'\s+([.,!?;:—\-])\s+', ' ', text)
    return text

In [13]:
apply_and_check(df, clean_text_keyword, 'text', 'clean_with_stopwords') 
print()

apply_and_check(df, clean_text_keyword, 'text', 'clean_with_stopwords_with_casing', lower=False)

Символов до: 1,036,674, после: 938,817
Удалено символов: 97,857 (9.44%)

Символов до: 1,036,674, после: 938,817
Удалено символов: 97,857 (9.44%)


### Орфография

Используем Hunspell, то есть spylls, который то же самое, но новее.  
Может стоило использовать [JamSpell](https://github.com/bakwc/JamSpell), который намного быстрее и точнее, но это только на Linux, а для Windows,  версия JamSpellPro, просто не открывается сайт с инструкциями.  
Можно попробовать также C++ через обёртку Cynthon с помощью CyHunspell, но оно при установке выдаёт ошибку. Кажется оно не работает на python 3.12, только [3.10](https://pypi.org/project/cyhunspell-py310/).  
Ещё можно попробовать SymSpell, который будет поновее и быстрее во много раз со своими фичами.  
Так же попробуем PySpellChecker.

In [14]:
from tqdm import tqdm
import time

def print_spell_check_stats(stats, elapsed_time):
    corrected_words_percent = stats['corrected_words'] / stats['total_words'] * 100
    speed = stats['total_words'] / elapsed_time
    
    print(f"Слов проверено: {stats['total_words']:,}")
    print(f"Исправлено слов: {stats['corrected_words']:,} ({corrected_words_percent:.2f})%, "
          f"уникальных слов: {len(stats['unique_words']):,}")
    print(f"Попаданий в кэш: {stats['cached_hits']:,}")
    print(f"Скорость: {speed:.0f} слов/сек")

#### Spylls

In [15]:
%%script cmd /c ""
# Добавил там, где неудачные методы, чтобы пропускать ячейки, если буду нажимать "Run All Cells"

from spylls.hunspell import Dictionary

# Скачал словарик мозиллы [https://addons.mozilla.org/ru/firefox/addon/russian-spellchecking-dic-3703/]
dictionary = Dictionary.from_zip('dictionary/russian_spellchecking_dic_3703-0.4.5.2resigned1.xpi')

word_cache = {}
stats = {
    'total_words': 0,
    'corrected_words': 0,
    'cached_hits': 0,
    'start_time': None,
}

def spell_check_spylls(text):
    # Вдруг попадётся не строка или нуль
    if not isinstance(text, str) or not text.strip():
        return text

    words = text.split()
    corrected_words = []
    
    for word in words:
        # Проверка на пустые слова или слова только из пробелов (можно убрать позже, если выявится, что слова не склеены случайно)
        if not word.strip():
            corrected_words.append(word)
            continue

        stats['total_words'] += 1
        
        # Проверяем кэш первым
        if word in word_cache:
            stats['cached_hits'] += 1
            corrected_words.append(word_cache[word])
            continue
        
        # Если нет в кэше - проверяем и кэшируем результат
        if dictionary.lookup(word):
            word_cache[word] = word
            corrected_words.append(word)
        else:
            suggestions = list(dictionary.suggest(word))
            corrected = suggestions[0] if suggestions else word

            # Если слово было исправлено
            if corrected != word:
                stats['corrected_words'] += 1
            
            word_cache[word] = corrected
            corrected_words.append(corrected)
    
    return ' '.join(corrected_words)

In [16]:
%%script cmd /c ""
stats['start_time'] = time.time()

tqdm.pandas(desc="Spylls")
df['corrected'] = df['clean'].progress_apply(spell_check_spylls) 
# Долго, первые 5 строк делает 6.7 секунды, без кэша ещё в 5x медленее.
# А мультипроцессорность и мультипоточность не помогает, скорее всего из-за GIL и то, что работаем в ipynb файле

elapsed_time = time.time() - stats['start_time']
print_spell_check_stats(stats, elapsed_time)

#### SymSpell

```python 
sym_spell.load_dictionary('dictionary/ru-100k.txt', term_index=0, count_index=1)
```
Не хочет нормально открывать словарь и выдаёт слово "Рё" на каждое короткое слово, а другие не исправляет, перед этим надо исправить txt файл, так как изначально кодировка utf-8, а стандартная кодировка для моей операционной системы для python - windows-1251 и таким образом не получается прочитать файл в кодировке utf-8.  
Можно обойти через смену стандартной кодировки или пересозданием объекта словаря.

In [17]:
from symspellpy import SymSpell, Verbosity

entries = []
with open('dictionary/ru-100k.txt', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        entries.append((parts[0].lower(), int(parts[1])))

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
for word, freq in entries:
    sym_spell.create_dictionary_entry(word, freq)

In [18]:
word_cache = {}
stats = {
    'total_words': 0,
    'corrected_words': 0,
    'cached_hits': 0,
    'start_time': None,
    'unique_words': set() # Можно через set, чтобы не дублировать слова
}
min_len = 3 # SymSpell пытается исправить множество мелких слов, даже если это не ошибка по типу ип, пао и подобное. Ограничим

def spell_check_symspell(text):
    if not isinstance(text, str) or not text.strip():
        return text
    
    words = text.split()
    corrected_words = []

    for word in words:
        stats['total_words'] += 1

        if len(word) <= min_len:
            corrected = word
        elif word in word_cache:
            stats['cached_hits'] += 1
            corrected = word_cache[word]
        else:
            suggestions = sym_spell.lookup(word.lower(), Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
            if suggestions:
                corrected = suggestions[0].term
            else:
                corrected = word
            word_cache[word] = corrected

        if corrected != word:
            stats['corrected_words'] += 1
            stats['unique_words'].add((word, corrected))

        corrected_words.append(corrected)
        
    return ' '.join(corrected_words)

In [19]:
stats['start_time'] = time.time()

tqdm.pandas(desc="SymSpell")
df['corrected'] = df['clean'].progress_apply(spell_check_symspell) 

elapsed_time = time.time() - stats['start_time']
print_spell_check_stats(stats, elapsed_time)

SymSpell: 100%|██████████| 1000/1000 [00:00<00:00, 2881.87it/s]

Слов проверено: 89,234
Исправлено слов: 8,017 (8.98)%, уникальных слов: 1,578
Попаданий в кэш: 71,570
Скорость: 254228 слов/сек


#### PySpellChecker

In [20]:
%%script cmd /c ""
from spellchecker import SpellChecker

spell_checker = SpellChecker(language='ru')

word_cache = {}
stats = {
    'total_words': 0,
    'corrected_words': 0,
    'cached_hits': 0,
    'start_time': None,
}

def spell_check_py(text):
    if not isinstance(text, str) or not text.strip():
        return text
    
    words = text.split()
    corrected_words = []
    
    for word in words:
        if not word.strip():
            corrected_words.append(word)
            continue
        
        stats['total_words'] += 1
        
        if word in word_cache:
            stats['cached_hits'] += 1
            corrected_words.append(word_cache[word])
            continue
        
        if word not in spell_checker.unknown([word]):
            word_cache[word] = word
            corrected_words.append(word)
        else:
            corrected = list(spell_checker.correction(word))[0] if spell_checker.correction(word) else word
            
            if corrected != word:
                stats['corrected_words'] += 1
            
            word_cache[word] = corrected
            corrected_words.append(corrected)
    
    return ' '.join(corrected_words)

In [21]:
%%script cmd /c ""
stats['start_time'] = time.time()

tqdm.pandas(desc="pyspellchecker")
df['corrected'] = df['clean'].progress_apply(spell_check_py)

elapsed_time = time.time() - stats['start_time']
print_spell_check_stats(stats, elapsed_time)

Если проверять орфографию на сыром датасете без кэша, то выйдет ~36 минут при скорости 60 слов в секунду. Примерная [таблица сравнения](https://github.com/diffitask/spell-checkers-comparison)

#### Итого исправления орфографии

Вывел сюда output, если выход ячеек сверху потеряется

```python
pyspellchecker: 100%|██████████| 1000/1000 [35:20<00:00,  2.12s/it]
Всего слов обработано: 88,529
Исправлено уникальных слов: 2,741
Попаданий в кэш: 81,487
Скорость: 42 слов/сек
```
=======================================================
```python
SymSpell: 100%|██████████| 1000/1000 [00:00<00:00, 2826.62it/s]
Слов проверено: 87,982
Исправлено слов: 8,017 (9.11)%, уникальных слов: 1,578
Попаданий в кэш: 70,319
Скорость: 245972 слов/сек
```
=======================================================
```python
Spylls: 100%|██████████| 1000/1000 [16:24<00:00,  1.02it/s]
Всего слов обработано: 88,651
Исправлено уникальных слов: 2,245
Попаданий в кэш: 81,579
Скорость: 90 слов/сек
```

Статистика по Spylls и pyspellchecker старая, так как работают 16 и 35 минут соотвественно, нету смысла повторять их из-за скорости.  
В итоге SymSpell сработал намного быстрее

### Лемматизация

In [22]:
# Сделаем все слова в именительном падеже с помощью лемматизации для лучшей обработки
# Также обработаем отдельно слова с "не" как "не_слово" так как много слово по типу "не удовлетворительно", а nltk stopwords убирает "не"
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

def lemmatize_text(text):
    raw_tokens = text.split()

    lemmas = []
    for token in raw_tokens:
        token = token.strip()
        if token and token != 'не':
            normal_form = morph.parse(token)[0].normal_form
            lemmas.append(normal_form)
        else:
            lemmas.append(token)

    # Склеиваем "не" с последующими леммами
    result = []
    i = 0
    while i < len(lemmas):
        if lemmas[i] == 'не' and i + 1 < len(lemmas):
            # Соединяем лемму "не" со следующей леммой
            result.append('не_' + lemmas[i + 1])
            i += 2 # Пропускаем оба токена
        else:
            result.append(lemmas[i])
            i += 1
    
    return ' '.join(result)

In [23]:
df['lemmatized_text'] = df['corrected'].apply(lemmatize_text)

In [24]:
df['lemmatized_text'].iloc[5:16] # Самые короткие строки в начале датасета

5                                                                                                                                                                                                                                                                          рамка данный ип установить отношение недвижимый имущество земельный участок республика северный осетия албания р н кировский ст ца земский ул маяковский кадастровый номер здание республика северный осетия албания кировский район ст земский маяковский ул далее кадастровый номер вынести постановление запрет действие
6                                                                                                                                                                                                                                                  взыскивать денежный средство перечислять следующий реквизит получатель ооо мкк верить расчётный получатель инна ооо мкк верить кпп получатель ооо мкк верить инна кпп короче з

## Выделение ключевых слов

### RAKE метод

Применяется к обычному тексту, как так RAKE работает от стоп-слов

In [25]:
from rake_nltk import Rake

rake = Rake(language='russian',
            min_length=2, # минимальное количество слов в фразе
            max_length=10 # максимальное количество слов в фразе
            )

def extract_keywords_rake(text, top_n=5):
    rake.extract_keywords_from_text(text)
    
    # Получаем топ-N фраз
    phrases = rake.get_ranked_phrases()[:top_n]
    
    # Возвращаем как строку
    return ' | '.join(phrases) if phrases else 'Нет ключевых слов'

In [26]:
df_rake = df.copy()
df_rake['rake'] = df_rake['clean_with_stopwords'].apply(lambda x: extract_keywords_rake(x, 5))

df_rake[['clean_with_stopwords', 'rake']].head(5)

,clean_with_stopwords,rake
0,"обратить взыскание на денежные средства должника ковалев вячеслав геннадьевич г.р., находящиеся на счетах в банках и иных кредитных организациях: юго-западный банк пао сбербанк, бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: филиал no банка втб (пао), бик счет: филиал no банка втб (пао), бик счет: филиал no банка втб (пао), бик счет: юго-западный банк пао сбербанк, бик счет: филиал no банка втб (пао), бик счет: филиал no банка втб (пао), бик счет: филиал no банка втб (пао), бик счет: кб ""ренессанс кредит"" (ооо), бик счет: пао сбербанк, бик счет: пао сбербанк, бик счет: пао сбербанк, бик счет: пао сбербанк, бик счет: пао сбербанк, бик счет: пао росбанк, бик счет: курское отделение n пао сбербанк, бик счет: ао ""альфа-банк"", бик счет: банк гпб (ао), бик счет: банк гпб (ао), бик счет: филиал центральный пао банка ""фк открытие"", бик счет: филиал no банка втб (пао), бик счет: пао ""промсвязьбанк"", бик счет: независимо от остатка денежных средств на счете.","денежные средства должника ковалев вячеслав геннадьевич г | курское отделение n пао сбербанк | фк открытие "", бик счет | филиал no банка втб | филиал no банка втб"
1,"обратить взыскание на денежные средства должника арбузова михаила михайловича г.р., находящиеся на счетах в банках и иных кредитных организациях: ао ""тинькофф банк"", бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: ао ""тинькофф банк"", бик счет: акб ""абсолют банк"" (пао), бик счет: ;",денежные средства должника арбузова михаила михайловича г | оренбургское отделение n пао сбербанк | оренбургское отделение n пао сбербанк | оренбургское отделение n пао сбербанк | оренбургское отделение n пао сбербанк
2,"судебным приставом-исполнителем обращено взыскание на денежные средства должника кочетова сергея николаевича г.р., находящиеся на счетах в банках и иных кредитных организациях: волгоградское отделение пао сбербанк, бик счет: южный филиал ао ""райффайзенбанк"", бик счет: ао ""тинькофф банк"", бик счет: ао ""тинькофф банк"", бик счет: ао ""тинькофф банк"", бик счет: ао ""райффайзенбанк"", бик счет: пао сбербанк, бик счет: кб ""акрополь"" ао, бик счет: филиал no банка втб (пао), бик счет: южный филиал ао ""райффайзенбанк"", бик счет: южный филиал ао ""райффайзенбанк"", бик счет: волгоградское отделение пао сбербанк, бик счет: волгоградское отделение пао сбербанк, бик счет: волгоградское отделение пао сбербанк, бик счет: банк гпб (ао), бик счет: банк гпб (ао), бик счет: банк гпб (ао), бик счет: банк гпб (ао), бик счет: банк гпб (ао), бик счет: филиал no банка втб (пао), бик счет: ;.","денежные средства должника кочетова сергея николаевича г | тинькофф банк "", бик счет | тинькофф банк "", бик счет | тинькофф банк "", бик счет | райффайзенбанк "", бик счет"
3,". вынсено постановление на запрет рег.действий с тс (не указано; не указано; газ г.в.; г/н кан; номер шасси (рамы) номер кузова (прицепа) № двиг: объём двигателя, см куб. .; мощность двигателя, квт .; мощность двигателя, л.с. .; первичный птс (уникальный номер регистрации) ом; наименование технологической операции изменение собственника (владельца); наименование регистрационного документа свидетельство о регистраци

Метод выдаёт самые часто встречающиеся слова, не учитывает важность некоторых слов в начале и конце, хоть и работает со стоп-словами. Даже после лёгкой очистки от шума. То есть не подходит для этого датасета и лучше работает на неформальном неповторяющемся тексте.  
Также можно было вместо удаления частых слов, попробовать найти именованные сущности через **NER** и заменить их на единый токен **[BANK]** или убрать банк, счёт и так далее повторяющееся вовсе. Также можно было попробовать убрать часто встречающиеся фразы в датасете через просмотр через tf-idf, но тут будто дальше тупик.

### YAKE метод

In [27]:
import yake

yake_extractor = yake.KeywordExtractor (
    lan = "ru",     
    n = 10,         # максимальное количество слов в фразе
    dedupLim = 0.8, # порог похожести слов
    top = 3         # количество ключевых слов
    )

df['yake'] = df['clean_with_stopwords_with_casing'].apply(
    # lambda x: [kw for kw, score in yake_extractor.extract_keywords(x)]
    lambda x: '; '.join(kw for kw, score in yake_extractor.extract_keywords(x))
)

df[['clean_with_stopwords_with_casing', 'yake']].head(5)

,clean_with_stopwords_with_casing,yake
0,"Обратить взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ ГЕННАДЬЕВИЧ г.р., находящиеся на счетах в банках и иных кредитных организациях: ЮГО-ЗАПАДНЫЙ БАНК ПАО СБЕРБАНК, БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ЮГО-ЗАПАДНЫЙ БАНК ПАО СБЕРБАНК, БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: КБ ""Ренессанс Кредит"" (ООО), БИК счет: ПАО СБЕРБАНК, БИК счет: ПАО СБЕРБАНК, БИК счет: ПАО СБЕРБАНК, БИК счет: ПАО СБЕРБАНК, БИК счет: ПАО СБЕРБАНК, БИК счет: ПАО РОСБАНК, БИК счет: КУРСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: АО ""АЛЬФА-БАНК"", БИК счет: БАНК ГПБ (АО), БИК счет: БАНК ГПБ (АО), БИК счет: ФИЛИАЛ ЦЕНТРАЛЬНЫЙ ПАО БАНКА ""ФК ОТКРЫТИЕ"", БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: независимо от остатка денежных средств на счете.",Обратить взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ ГЕННАДЬЕВИЧ; взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ ГЕННАДЬЕВИЧ; Обратить взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ
1,"Обратить взыскание на денежные средства должника Арбузова Михаила Михайловича г.р., находящиеся на счетах в банках и иных кредитных организациях: АО ""ТИНЬКОФФ БАНК"", БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: АО ""ТИНЬКОФФ БАНК"", БИК счет: АКБ ""АБСОЛЮТ БАНК"" (ПАО), БИК счет: ;",Обратить взыскание на денежные средства должника Арбузова Михаила Михайловича; взыскание на денежные средства должника Арбузова Михаила Михайловича; Обратить взыскание на денежные средства должника Арбузова Михаила
2,"Судебным приставом-исполнителем обращено взыскание на денежные средства должника Кочетова Сергея Николаевича г.р., находящиеся на счетах в банках и иных кредитных организациях: ВОЛГОГРАДСКОЕ ОТДЕЛЕНИЕ ПАО СБЕРБАНК, БИК счет: ЮЖНЫЙ ФИЛИАЛ АО ""РАЙФФАЙЗЕНБАНК"", БИК счет: АО ""ТИНЬКОФФ БАНК"", БИК счет: АО ""ТИНЬКОФФ БАНК"", БИК счет: АО ""ТИНЬКОФФ БАНК"", БИК счет: АО ""РАЙФФАЙЗЕНБАНК"", БИК счет: ПАО СБЕРБАНК, БИК счет: КБ ""АКРОПОЛЬ"" АО, БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ЮЖНЫЙ ФИЛИАЛ АО ""РАЙФФАЙЗЕНБАНК"", БИК счет: ЮЖНЫЙ ФИЛИАЛ АО ""РАЙФФАЙЗЕНБАНК"", БИК счет: ВОЛГОГРАДСКОЕ ОТДЕЛЕНИЕ ПАО СБЕРБАНК, БИК счет: ВОЛГОГРАДСКОЕ ОТДЕЛЕНИЕ ПАО СБЕРБАНК, БИК счет: ВОЛГОГРАДСКОЕ ОТДЕЛЕНИЕ ПАО СБЕРБАНК, БИК счет: БАНК ГПБ (АО), БИК счет: БАНК ГПБ (АО), БИК счет: БАНК ГПБ (АО), БИК счет: БАНК ГПБ (АО), БИК счет: БАНК ГПБ (АО), БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ;.",приставом-исполнителем обращено взыскание на денежные средства должника Кочетова Сергея Николаевича; Судебным приставом-исполнителем обращено взыскание на денежные средства должника Кочетова Сергея; обращено взыскание на денежные средства должника Кочетова Сергея Николаевича
3,". вынсено постановление на запрет рег.действий с ТС (Не указано; Не указано; ГАЗ г.в.; г/н КАН; Номер шасси (рамы) Номер кузова (прицепа) № двиг: Объём двигателя, см куб. .; Мощность двигателя, кВт .; Мощность двигателя, л.с. .; Первичный ПТС (уникальный номер

YAKE же сработал намного лучше, показывая начало текста и конец, но также выделяет лишние слова.  
YAKE состоит из 5 разных метрик:  
- Casing (регистр) - В нашем случае важно, связано с началом предложений
- Word Position (позиция, начало и конец предложения) - То же самое, так как основной текст в начале и чуть в конце
- Word Frequency (частота) - Будто метрика только портит из-за счёт, банк, перечисление имущества
- Word Relatedness to Context (стоп-слова) - Как и RAKE, может помочь, а может нет
- DifSentence (частота появления в предложениях) - Так как текст является формальным и кратким, может быть не очень верным

Хоть и не пробовал метод TextRank, в итоге нету смысла от выделения ключевых слов, так как уже лемматизированные данные подходят под rule-based метод, чтобы определить часть answer, да и задача классификации/генерации не для выделения ключевых слов

## Классификация

### Rule метод

Так как в тексте могут быть явные указатели, при этом ответ почти всегда следует одному шаблону, запишем в answer уже явные ответы

In [100]:
import re

# Функции чтобы искать именно слова через "re", а не символы
# Хотя бы одно слово
def has_any_words(words, text):
    if not words:
        return False
    pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    return bool(re.search(pattern, text, flags=re.IGNORECASE))

# Хотя бы все слова
def has_all_words(words, text):
    for word in words:
        pattern = r'\b' + re.escape(word) + r'\b'
        if not re.search(pattern, text, flags=re.IGNORECASE):
            return False
    return True

# Тут я пытался сделать через список, функции, сочетание с lambda этих способов, 
# но ничего не работало из-за логики определения удовлетворено/частично удовлетворено/отказ и без if elif это не сделать
# Пока что это тяжело читать и расширять, но это делается легко и быстро. То есть это не в моего опыта сделать это лучше
def extract_status(text):
    if has_all_words(['подлежащее удовлетворение'], text):
        if has_any_words(['не_подлежащее', 'не_удовлетворение'], text):
            return 'Частично удовлетворено'
        return 'Удовлетворено'
    elif has_any_words(['не_подлежащее', 'не_удовлетворение'], text):
        if has_all_words(['подлежащее', 'удовлетворение'], text):
            return 'Частично удовлетворено'
        return 'Отказ'
    elif has_all_words(['обратить', 'взыскание'], text) or has_all_words(['обращение', 'взыскание',], text) and not 'не_иметься' in text:
        return 'Взыскание обращено'
    elif has_any_words(['действие', 'рег', 'регистрационный'], text) and 'запрет' in text:
        return 'Запрет действий'
    elif has_all_words(['невозможно', 'установить'], text) or has_all_words(['не_представиться', 'лицо'], text) or has_all_words(['не_установить', 'должник'], text):
        return 'Невозможно установить местонахождение должника'
    # elif has_all_words(['окончить', 'производство'], text): # Слишком малый класс - 5 примеров
    #     return 'Производство окончено'
    elif has_all_words(['вынести', 'постановление'], text):
        return 'Постановление вынесено'
    elif has_all_words(['направить', 'запрос'], text):
        return 'Запрос направлен'
    elif has_all_words(['обращение', 'рассмотреть'], text) or has_all_words(['обращение', 'ответ'], text):
        return 'Обращение рассмотрено'
    else:
        return 'Подробности не известны'

In [101]:
df['answer'] = df['lemmatized_text'].apply(extract_status)

In [102]:
print(df['answer'].value_counts())
print("Всего обработано строк:", df['answer'].value_counts().sum())

answer
Подробности не известны                           315
Отказ                                             195
Частично удовлетворено                            116
Взыскание обращено                                108
Обращение рассмотрено                             100
Запрос направлен                                   78
Постановление вынесено                             36
Удовлетворено                                      27
Запрет действий                                    15
Невозможно установить местонахождение должника     10
Name: count, dtype: int64
Всего обработано строк: 1000


In [103]:
# df[['answer','lemmatized_text']].iloc[610:614]
df[['answer','lemmatized_text']].iloc[354:357]

,answer,lemmatized_text
354,Подробности не известны,судебный пристав исполнитель рамка исполнительный производство отношение должник розыск должник имущество не_представляться возможный связь необходимость проведение доразыскной мероприятие направление запрос гу гибрид наличие административный правонарушение связанный участие данный транспорт предоставление информация передвижение автотранспортный средство также проверить фактпроживание факт нахождение тс весь известный адрес должник выход адрес должник запланировать октябрь
355,Удовлетворено,поступить ходатайство осуществить выход местонахождение имущество должник перепечин дмитрий юриевич цель совершение исполнительный действие часть арест имущество находиться адрес проживание регистрация временной пребывание должник рассмотреть довод заявитель техник владимир юриевич признать обстоятельство изложить заявление ходатайство подлежащее удовлетворение
356,Подробности не известны,судебный пристать исполнитель пушкинский рост гуфссп московский область подразделение проезд й фабричный далее пушкин московский обл андреев елена иванович рассмотреть материал исполнительный производство ип возбудить основание исполнительный документ постановление выдать орган ивантеевский городской суд дело вступить законный сила предмет исполнение административный приостановление деятельность срок сутки отношение ип марченко отношение должник марченко александр сергеевич сниться год рождение должник ул колпаков бытие московский обл польза взыскатель гу мвд московский область взыскатель ул никитинский далее москва установить


Таким образом осталось ~треть данных из 1000 строк  
Остались только не входящие в правила строки и, возможно, неправильно обработанные

Также стоит убрать классы меньше чем с 10 примерами, так как они позже будут мешать обучению модели, а так при их удалении позже по коду не могу исправить ошибку с индексами, что я и сделал, закомменировав код сверху в правилах.  
Если же переименовать такие классы на просто "другие", то модель может классифицировать многие тексты, как "другие", что не очень то является ответом на заявление

##### Через множество соединённых классов

In [104]:
%%script cmd /c ""
import re

# Функции чтобы искать именно слова через "re", а не символы
# Хотя бы одно слово
def has_any_words(words, text):
    if not words:
        return False
    pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    return bool(re.search(pattern, text, flags=re.IGNORECASE))

# Хотя бы все слова
def has_all_words(words, text):
    for word in words:
        pattern = r'\b' + re.escape(word) + r'\b'
        if not re.search(pattern, text, flags=re.IGNORECASE):
            return False
    return True

def extract_status(text):
    statuses = []

    if has_all_words(['подлежащее удовлетворение'], text):
        statuses.append('Удовлетворено')
    if has_any_words(['не_подлежащее', 'не_удовлетворение'], text):
        if has_all_words(['подлежащее', 'удовлетворение'], text):
            statuses.append('Частично удовлетворено')
        statuses.append('Отказ')
    if has_all_words(['ип', 'приостановлено'], text):
        statuses.append('ИП приостановлено')
    if has_any_words(['приостановить'], text):
        statuses.append('Приостановлено')
    if has_all_words(['окончить', 'производство'], text):
        statuses.append('Производство окончено')
    if has_all_words(['обратить', 'взыскание'], text) or has_all_words(['обращение', 'взыскание'], text):
        statuses.append('Взыскание обращено')
    if has_any_words(['действие', 'рег', 'регистрационный'], text) and 'запрет' in text:
        statuses.append('Запрет действий')
    if has_all_words(['невозможно', 'установить'], text) or has_any_words(['не_установить', 'розыск'], text):
        statuses.append('Отказ, невозможно установить местонахождение должника')
    if has_all_words(['вынести', 'постановление'], text):
        statuses.append('Постановление вынесено')
    if has_all_words(['направить', 'запрос'], text):
        statuses.append('Запрос направлен')
    
    return statuses if statuses else None

In [105]:
%%script cmd /c ""
df['answer'] = df['lemmatized_text'].apply(extract_status)
print(df['answer'].value_counts())
print("Всего обработано строк:", df['answer'].value_counts().sum())

```python
answer
[Отказ]                                                                                                         130
[Запрос направлен]                                                                                               66
[Удовлетворено, Частично удовлетворено, Отказ]                                                                   66
[Отказ, невозможно установить местонахождение должника]                                                          41
[Взыскание обращено]                                                                                             35
                                                                                                               ... 
[Удовлетворено, Частично удовлетворено, Отказ, Взыскание обращено, Постановление вынесено, Запрос направлен]      1
[Отказ, Взыскание обращено, Отказ, невозможно установить местонахождение должника]                                1
[Отказ, Запрет действий, Постановление вынесено, Запрос направлен]                                                1
[Отказ, Взыскание обращено, Запрет действий, Постановление вынесено]                                              1
[Частично удовлетворено, Отказ]                                                                                   1
Name: count, Length: 62, dtype: int64
Всего обработано строк: 625
```
То есть вышло 87 классов через множество соединённых классов, что не вариант на этом датасете

##### Итог

Пока разметка классов идёт по первым встреченным словам.  
Проблема в том, что одна строка может иметь много классов, что переходит в многолейбловую классификацию, что не оставляет самые распространённые методы по типу SVM, Log Reg, K-means, а требует что-то похожее на MultiOutputClassifier.  
Также среди размеченных данных могут быть такие классы, которые встречаются только в числе других классов, а не одиночно, что значит не хватает размера датасета и модель просто не сможет правильно классифицировать.  

Можно сделать отдельные классы для совмещенных "Удовлетворительно + Взыскание обращено", но тогда классов будет намного больше, а датасет всё такой же маленький

### Naive Bayes, Logistic Regression, SVM, RandomForest, GradientBoosting, CatBoost, XGBoost, LightGBM

Попробуем линейные модели, если они будут плохи, откинем их.  
Скорее всего бустинг модели будут хороши в этой задаче, а может и слишком сложны.  
Разница между ними под капотом большая, но можно попробовать все, сравнить результаты.

Обучим алгоритмы на тех данных, которые уже имеют верные метки

In [106]:
df_labeled = df[df['answer'].notna()].copy()
# original_indices_labeled = df_labeled.index.copy() # Для последующего объеденения с предсказаниями
df_labeled = df_labeled.reset_index(drop=True)

df_labeled['answer'].value_counts()

answer
Подробности не известны                           315
Отказ                                             195
Частично удовлетворено                            116
Взыскание обращено                                108
Обращение рассмотрено                             100
Запрос направлен                                   78
Постановление вынесено                             36
Удовлетворено                                      27
Запрет действий                                    15
Невозможно установить местонахождение должника     10
Name: count, dtype: int64

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = df_labeled['lemmatized_text']
y = df_labeled['answer']

# Может стоит попробовать Stratified K-Fold, раз классы не сбалансированы, посмотреть как как модели стабильны
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=1,
    stratify=y
)

# Преобразуем текстовые метки в числовые для бустинг методов (XGBoost просит)
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

In [109]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

classifiers = [
    ('Naive Bayes', MultinomialNB()),
    ('Logistic Regression', LogisticRegression(max_iter=1000, 
                                               random_state=1, 
                                               class_weight='balanced')), # балансируем, так как слишком редкие классы
    ('SVM', LinearSVC(max_iter=2000, 
                      random_state=1, 
                      class_weight='balanced')),
    ('Random Forest', RandomForestClassifier(max_depth=6, # попытался ограничить глубину деревьев, так как кажется слишком сложные модели для такой задачи
                                             random_state=1,
                                             class_weight='balanced')),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=1)),
    ('CatBoost', CatBoostClassifier(verbose=0,
                                    random_state=1,
                                    n_estimators=100,
                                    max_depth=6,
                                    loss_function='MultiClass',
                                    auto_class_weights='Balanced',
                                    train_dir=None)), # всё равно сохраняет папку
    ('XGBoost', XGBClassifier(objective='multi:softmax', 
                              random_state=1, 
                              n_estimators=100,
                              learning_rate=0.1,
                              max_depth=6,
                              num_class=len(le.classes_))),
    ('LightGBM', LGBMClassifier(objective='multiclass',
                                n_estimators=100,
                                max_depth=6,
                                num_class=len(le.classes_),
                                class_weight='balanced',
                                random_state=1,
                                verbosity=-1))
]

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

for name, classifier in classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")
    
    # Создаем pipeline для каждой модели
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000,
                                ngram_range=(1, 2))),
        ('classifier', classifier)
    ])
    
    pipeline.fit(X_train, y_train_enc)
    y_pred = pipeline.predict(X_test)
    y_pred_labels = le.inverse_transform(y_pred)
    
    print(classification_report(y_test, y_pred_labels, digits=3))


Модель: Naive Bayes
                                                precision    recall  f1-score   support

                            Взыскание обращено      0.607     0.531     0.567        32
                               Запрет действий      0.000     0.000     0.000         4
                              Запрос направлен      0.600     0.261     0.364        23
Невозможно установить местонахождение должника      0.000     0.000     0.000         3
                         Обращение рассмотрено      0.800     0.133     0.229        30
                                         Отказ      0.826     0.966     0.891        59
                       Подробности не известны      0.603     0.958     0.740        95
                        Постановление вынесено      0.000     0.000     0.000        11
                                 Удовлетворено      0.000     0.000     0.000         8
                        Частично удовлетворено      0.811     0.857     0.833        35

         

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

                                                precision    recall  f1-score   support

                            Взыскание обращено      0.733     0.688     0.710        32
                               Запрет действий      1.000     0.250     0.400         4
                              Запрос направлен      0.483     0.609     0.538        23
Невозможно установить местонахождение должника      0.400     0.667     0.500         3
                         Обращение рассмотрено      0.793     0.767     0.780        30
                                         Отказ      0.982     0.949     0.966        59
                       Подробности не известны      0.804     0.779     0.791        95
                        Постановление вынесено      0.077     0.091     0.083        11
                                 Удовлетворено      0.700     0.875     0.778         8
                        Частично удовлетворено      1.000     0.971     0.986        35

                              

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

                                                precision    recall  f1-score   support

                            Взыскание обращено      0.969     0.969     0.969        32
                               Запрет действий      1.000     0.500     0.667         4
                              Запрос направлен      0.500     0.652     0.566        23
Невозможно установить местонахождение должника      0.000     0.000     0.000         3
                         Обращение рассмотрено      0.684     0.867     0.765        30
                                         Отказ      0.983     1.000     0.992        59
                       Подробности не известны      0.856     0.811     0.832        95
                        Постановление вынесено      0.667     0.364     0.471        11
                                 Удовлетворено      1.000     1.000     1.000         8
                        Частично удовлетворено      1.000     0.971     0.986        35

                              

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\

                                                precision    recall  f1-score   support

                            Взыскание обращено      0.789     0.938     0.857        32
                               Запрет действий      1.000     1.000     1.000         4
                              Запрос направлен      0.720     0.783     0.750        23
Невозможно установить местонахождение должника      0.000     0.000     0.000         3
                         Обращение рассмотрено      0.824     0.933     0.875        30
                                         Отказ      0.983     0.966     0.974        59
                       Подробности не известны      0.925     0.905     0.915        95
                        Постановление вынесено      0.667     0.364     0.471        11
                                 Удовлетворено      1.000     1.000     1.000         8
                        Частично удовлетворено      0.971     0.943     0.957        35

                              

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMe

`UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use zero_division parameter to control this behavior. _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])`

Ошибка возникает, так как модель Naive Bayes не предсказал ни одного примера редких классов, SVM и Log Reg же попытались, хоть и не очень удачно по precision, recall, f1, но немного лучше чем просто угадывание по всем классам

SVM и Log Reg имеют лучшие показатели точности accuracy, но в этом случае это не показывает качество модели из-за несбалансированных классов.
NB имеет худшие результаты, он не определяет самые редкие классы, впрочем SVM и Log Reg методы так же имеют плохие f1 score на тех же классах, хоть и показывают хорошие macro avg (метрика, которая вместе с несбалансированными классами больше показывает модель) и weighted avg.  

Модели Random Forest, Gradient Boosting, CatBoost, XGBoost и LightGBM уже имеют намного лучшие результаты, но, кажется, есть переобучение, особенно LightGBM, так как почти все метрики 1.0 по F1, даже на редких классах. Попробовал ограничить глубину.

### Вместе с Stratified K-Fold

#### Попытка 1

Попробуем проверить через K-Fold, но с Stratified, чтобы учесть несбалансированность классов и понять какие модели не нестабильны.

In [111]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for name, classifier in classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")
    
    # Создаем pipeline для каждой модели
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000,
                                ngram_range=(1, 2))),
        ('classifier', classifier)
    ])

    # Кросс-валидация
    y_pred_cv = cross_val_predict(
        pipeline,
        X_train,
        y_train_enc,
        cv=skf,
        method='predict'
    )

    y_pred_cv_labels = le.inverse_transform(y_pred_cv)
    y_train_labels = le.inverse_transform(y_train_enc)
    
    print(classification_report(y_train_labels, y_pred_cv_labels, digits=3))


Модель: Naive Bayes


c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

                                                precision    recall  f1-score   support

                            Взыскание обращено      0.506     0.553     0.528        76
                               Запрет действий      0.000     0.000     0.000        11
                              Запрос направлен      0.583     0.255     0.354        55
Невозможно установить местонахождение должника      0.000     0.000     0.000         7
                         Обращение рассмотрено      0.667     0.114     0.195        70
                                         Отказ      0.880     0.919     0.899       136
                       Подробности не известны      0.579     0.914     0.709       220
                        Постановление вынесено      0.000     0.000     0.000        25
                                 Удовлетворено      0.000     0.000     0.000        19
                        Частично удовлетворено      0.870     0.988     0.925        81

                              

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                                                precision    recall  f1-score   support

                            Взыскание обращено      0.818     0.947     0.878        76
                               Запрет действий      0.889     0.727     0.800        11
                              Запрос направлен      0.873     0.873     0.873        55
Невозможно установить местонахождение должника      0.000     0.000     0.000         7
                         Обращение рассмотрено      0.800     0.971     0.877        70
                                         Отказ      1.000     0.971     0.985       136
                       Подробности не известны      0.923     0.873     0.897       220
                        Постановление вынесено      0.810     0.680     0.739        25
                                 Удовлетворено      0.950     1.000     0.974        19
                        Частично удовлетворено      1.000     0.988     0.994        81

                              

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

                                                precision    recall  f1-score   support

                            Взыскание обращено      0.921     0.921     0.921        76
                               Запрет действий      1.000     0.455     0.625        11
                              Запрос направлен      0.870     0.855     0.862        55
Невозможно установить местонахождение должника      0.000     0.000     0.000         7
                         Обращение рассмотрено      0.857     0.943     0.898        70
                                         Отказ      0.993     0.978     0.985       136
                       Подробности не известны      0.907     0.973     0.939       220
                        Постановление вынесено      0.714     0.600     0.652        25
                                 Удовлетворено      1.000     0.895     0.944        19
                        Частично удовлетворено      0.988     0.975     0.981        81

                              

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


                                                precision    recall  f1-score   support

                            Взыскание обращено      0.960     0.947     0.954        76
                               Запрет действий      1.000     0.727     0.842        11
                              Запрос направлен      0.906     0.873     0.889        55
Невозможно установить местонахождение должника      0.000     0.000     0.000         7
                         Обращение рассмотрено      0.930     0.943     0.936        70
                                         Отказ      1.000     0.985     0.993       136
                       Подробности не известны      0.936     0.991     0.962       220
                        Постановление вынесено      0.833     0.800     0.816        25
                                 Удовлетворено      1.000     1.000     1.000        19
                        Частично удовлетворено      0.988     1.000     0.994        81

                              

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Новые метрики уже больше похожи на правду, почти нету метрик с идеальным показателем 1.0.  
Также линейные модели лучше предсказывают самый редкий класс, чем бустинговые модели, кроме Gradient Boosting. Это может быть, потому-что бустинг модели пытаются уменьшить ошибку на всех классах, пренебрегая самым редким классом спустя множество бустинг моделей.

#### Попытка 2

Можно попробовать увеличить размер минорного (миноритарного) класса через его дублирование (RandomOverSampler)(более простой) или через SMOTE.  
Также можно попробовать изменить вес класса вручную для каждой модели, чем и можно заняться перед SMOTE.  
И Focal Loss попробовать для поддерживаемых моделей.  

### Вместе с SMOTE и Stratified K-Fold

K-Fold показал, что всё таки требуется SMOTE, так как некоторые метрики не сильно растут, а другие снижаются.

Думал аугментировать данные, но замена синонимами, перефразированием в таком датасете может не представлять сам датасет, так как фразы шаблонны.  
Можно сделать синтетические примеры через LLM, но это напоследок, если всё будет плохо с малыми классами.

In [124]:
# Повторяем все импорты (и в ячейках выше тоже было), чтобы не активировать ячейки сверху и обучать модели по 5-10 минут, ожидая этой ячейки
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = df_labeled['lemmatized_text']
y = df_labeled['answer']

# Может стоит попробовать Stratified K-Fold, раз классы не сбалансированы
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=1,
    stratify=y # стратифицируем 1 раз из-за дисбаланса классов
)

# Преобразуем текстовые метки в числовые для бустинг методов (XGBoost просит)
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

In [122]:
# Посмотрим сколько будет пример до и после
from imblearn.over_sampling import SMOTE

vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)

smote = SMOTE(random_state=1, sampling_strategy='not majority')
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_tfidf, y_train)

print("Размер классов train до:", y_train.value_counts())
print("\nРазмер классов train после:", y_train_balanced.value_counts())

print("\nРазмер train до:", len(y_train))
print("Размер train после:", len(y_train_balanced))
print("Добавлено всего:", len(y_train_balanced) - len(y_train))

Размер классов train до: answer
Подробности не известны                           220
Отказ                                             136
Частично удовлетворено                             81
Взыскание обращено                                 76
Обращение рассмотрено                              70
Запрос направлен                                   55
Постановление вынесено                             25
Удовлетворено                                      19
Запрет действий                                    11
Невозможно установить местонахождение должника      7
Name: count, dtype: int64

Размер классов train после: answer
Подробности не известны                           220
Частично удовлетворено                            220
Удовлетворено                                     220
Обращение рассмотрено                             220
Запрос направлен                                  220
Отказ                                             220
Взыскание обращено                        

In [118]:
from imblearn.pipeline import Pipeline as ImbPipeline
# import pickle
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for name, classifier in classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")
    
    pipeline = ImbPipeline([
        ('tfidf', TfidfVectorizer(max_features=1000,
                                ngram_range=(1, 2))),
        ('smote', SMOTE(
            random_state=1,
            sampling_strategy='not majority',
            k_neighbors=4
        )),
        ('classifier', classifier)
    ])

    y_pred_cv = cross_val_predict(
        pipeline,
        X_train,
        y_train_enc,
        cv=skf,
        method='predict'
    )
    
    y_pred_cv_labels = le.inverse_transform(y_pred_cv)
    y_true_labels = le.inverse_transform(y_train_enc)
    
    print(classification_report(y_true_labels, y_pred_cv_labels, digits=3))

    # y_pred = pipeline.predict(X_test)
    # y_pred_labels = le.inverse_transform(y_pred)

    # print(classification_report(y_test, y_pred_labels))

    # trained_pipelines[name] = pipeline


    # with open(f'pipeline_{name}.pkl', 'wb') as f:
    #     pickle.dump(pipeline, f)


Модель: Naive Bayes
                                                precision    recall  f1-score   support

                            Взыскание обращено      0.656     0.553     0.600        76
                               Запрет действий      0.444     0.727     0.552        11
                              Запрос направлен      0.507     0.673     0.578        55
Невозможно установить местонахождение должника      0.062     0.143     0.087         7
                         Обращение рассмотрено      0.451     0.657     0.535        70
                                         Отказ      0.968     0.897     0.931       136
                       Подробности не известны      0.842     0.582     0.688       220
                        Постановление вынесено      0.225     0.360     0.277        25
                                 Удовлетворено      0.667     0.632     0.649        19
                        Частично удовлетворено      0.846     0.951     0.895        81

         

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                                                precision    recall  f1-score   support

                            Взыскание обращено      0.922     0.934     0.928        76
                               Запрет действий      0.889     0.727     0.800        11
                              Запрос направлен      0.815     0.800     0.807        55
Невозможно установить местонахождение должника      1.000     0.286     0.444         7
                         Обращение рассмотрено      0.800     0.914     0.853        70
                                         Отказ      1.000     0.963     0.981       136
                       Подробности не известны      0.906     0.918     0.912       220
                        Постановление вынесено      0.692     0.720     0.706        25
                                 Удовлетворено      0.850     0.895     0.872        19
                        Частично удовлетворено      1.000     0.963     0.981        81

                              

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

                                                precision    recall  f1-score   support

                            Взыскание обращено      0.945     0.908     0.926        76
                               Запрет действий      1.000     0.636     0.778        11
                              Запрос направлен      0.862     0.909     0.885        55
Невозможно установить местонахождение должника      0.000     0.000     0.000         7
                         Обращение рассмотрено      0.918     0.957     0.937        70
                                         Отказ      1.000     0.978     0.989       136
                       Подробности не известны      0.911     0.977     0.943       220
                        Постановление вынесено      0.826     0.760     0.792        25
                                 Удовлетворено      1.000     0.895     0.944        19
                        Частично удовлетворено      0.988     0.975     0.981        81

                              

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


                                                precision    recall  f1-score   support

                            Взыскание обращено      0.947     0.947     0.947        76
                               Запрет действий      1.000     0.727     0.842        11
                              Запрос направлен      0.911     0.927     0.919        55
Невозможно установить местонахождение должника      1.000     0.143     0.250         7
                         Обращение рассмотрено      0.971     0.957     0.964        70
                                         Отказ      1.000     0.971     0.985       136
                       Подробности не известны      0.928     0.991     0.958       220
                        Постановление вынесено      0.864     0.760     0.809        25
                                 Удовлетворено      1.000     1.000     1.000        19
                        Частично удовлетворено      0.988     1.000     0.994        81

                              

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


SMOTE помог только Naive Bayes методу предсказать ранее не предсказанные классы, и повысить все метрики.  
Метрики же на других методах не сильно поменялись или вообще стали чуть хуже. На более сложных же моделях метрики стали намного лучше 

В итоге лучшими моделями оказались Gradient Boosting, XGBoost и LightGBM.  
Можно сделать ансамбль из этих моделей

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = df_labeled['lemmatized_text']
y = df_labeled['answer']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=1,
    stratify=y
)

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

In [12]:
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

best_model_names = ['Gradient Boosting', 'XGBoost', 'LightGBM']
best_classifiers = [pair for pair in classifiers if pair[0] in best_model_names]

trained_pipelines = {}

for name, classifier in best_classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")

    pipeline = ImbPipeline([
        ('tfidf', TfidfVectorizer(max_features=1000,
                                ngram_range=(1, 2))),
        ('smote', SMOTE(
            random_state=1,
            sampling_strategy='not majority'
        )),
        ('classifier', classifier)
    ])
    
    pipeline.fit(X_train, y_train_enc)
    
    y_pred = pipeline.predict(X_test)
    y_pred_labels = le.inverse_transform(y_pred)
    print(classification_report(y_test, y_pred_labels, digits=3))

    trained_pipelines[name] = pipeline


Модель: Gradient Boosting
                        precision    recall  f1-score   support

    Взыскание обращено      0.914     1.000     0.955        32
       Запрет действий      1.000     1.000     1.000         5
      Запрос направлен      0.913     0.875     0.894        24
 Обращение рассмотрено      0.938     1.000     0.968        30
                 Отказ      1.000     1.000     1.000        59
Постановление вынесено      1.000     0.636     0.778        11
         Удовлетворено      1.000     1.000     1.000         8
Частично удовлетворено      1.000     1.000     1.000        35

              accuracy                          0.966       204
             macro avg      0.971     0.939     0.949       204
          weighted avg      0.967     0.966     0.964       204


Модель: XGBoost
                        precision    recall  f1-score   support

    Взыскание обращено      0.941     1.000     0.970        32
       Запрет действий      1.000     1.000     1.000   

c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


~~Так как метрики относительно одинаковы между Log Reg и SVM, можно выбрать одну из них. Однако Naive Bayes выдаёт иные метрики, некоторые лучше, в основном хуже. Можно попробовать ансамбль моделей~~

### Классификация неразмеченного текста

#### Через ансамбль

In [15]:
df_unlabeled = df[df['answer'].isna()].copy()
# original_indices_unlabeled = df_labeled.index.copy()
df_unlabeled = df_unlabeled.reset_index(drop=True) # Нужно обнулить индексы, так как предсказывает только 100 из 322 почему-то
X_unlabeled = df_unlabeled['lemmatized_text']

predictions_all = {}
for name, pipeline in trained_pipelines.items():
    preds = pipeline.predict(X_unlabeled)
    predictions_all[name] = preds.ravel() if hasattr(preds, "ravel") else preds # ravel() так как CatBoost выдаёт двумерный массив, вместо одномерного

predictions_df = pd.DataFrame(predictions_all)
# # Перевернуть столбцы, чтобы из-за mode, которая при всех разных вариантов выбирает первый столбце в списке, выбиралась лучшая модель - SVM
# predictions_df = predictions_df[['SVM', 'Logistic Regression', 'Naive Bayes']]

# Преобразуем обратно в текстовые метки
for col in predictions_df.columns:
    predictions_df[col] = le.inverse_transform(predictions_df[col])

# Выбрать наиболее частый ответ среди всех строк
df_unlabeled['answer'] = predictions_df.mode(axis=1)[0]


print("Датафрейм с предсказаниями от всех моделей:")
print(predictions_df.head())

print("\nКоличество предсказанных классов:", df_unlabeled['answer'].value_counts())

Датафрейм с предсказаниями от всех моделей:
       Gradient Boosting                XGBoost               LightGBM
0  Обращение рассмотрено  Обращение рассмотрено  Обращение рассмотрено
1  Обращение рассмотрено                  Отказ  Обращение рассмотрено
2  Обращение рассмотрено  Обращение рассмотрено  Обращение рассмотрено
3  Обращение рассмотрено        Запрет действий  Обращение рассмотрено
4     Взыскание обращено     Взыскание обращено     Взыскание обращено

Количество предсказанных классов: answer
Обращение рассмотрено     225
Запрос направлен           49
Взыскание обращено         40
Отказ                       3
Постановление вынесено      2
Частично удовлетворено      2
Запрет действий             1
Name: count, dtype: int64


c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [16]:
df_unlabeled[['answer', 'lemmatized_text']].head()

,answer,lemmatized_text
0,Обращение рассмотрено,взыскивать денежный средство перечислять следующий реквизит получатель ооо мкклый свериться расчётный получатель инна ооо мкклый свериться – кпд получатель ооо мкклый свериться инна кпд корсет занести исполнительный производство такой образ постановление действие судебный пристав исполнитель оп хунхузский распутинский тамильский район являться правомерный
1,Обращение рассмотрено,дать тю внести бдеть ас мобильный розыск дорожный пристать осуществляться еженедельный рейд совместно орган мрамор гибрид выявление залоговый тю направляться требование должник установление местонахождение залоговый тю дополнительно осуществляться выход предполагать место расположение тю также ежемесячно проверяться система отслеживание тю безопасный город
2,Обращение рассмотрено,согласно электронный ответ мрамор гибрид автотранспортный средство также сельскохозяйственный самоходный техника должник не_зарегистрировать согласно ответ возрождение «» «мдм » мниться » лето уральский смотреть каб полбанки скоба раструсить ооо «» каб экспресс волга райффайзенбанк россия – гс москва москва газпром гс ставрополь перегрев ставропольпромстройбанк должник не_являться клиент указанный кредитный учреждение открытый счёт
3,Обращение рассмотрено,согласно данные автоматизированный система контроль передвижение автотранспорт ас кап автотранспортный средство последний передвигаться спбгу ад эй километр ад внутренний кольцо съезд комендантский пр участок проезжий часть комендантский пр кольцевой автодорога парашютный ул зона контроль адрес указанный исполнительный документ должник не_установить осмотр домовой территория ст не_установить информация передвижение транспортный средство передать специализировать подразделение поток др установление транспортный средство послый ехать составление акт опись арест имущество
4,Взыскание обращено,алданский расти уфссп с сообщать исполнительный производство предмет исполнение взыскать солидарно должник маковый рт прокофиевич ню сидорович задолженность договор заём а ля рт окончить гс фактический исполнение один несколько должник требование солидарный взыскание содержаться исполнительный документ объединить сводный исполнительный производство


То есть любые неизвестные текста модели относят к "Обращение рассмотрено", что говорит о том, что классов недостаточно и нужно придумывать что-то иное, но, по-крайне мере модели предсказывают классы с высокими метриками

#### Итоговый датасет

In [17]:
df_labeled.index = df[df['answer'].notna()].index
df_unlabeled.index = df[df['answer'].isna()].index

In [18]:
df_result = pd.concat([df_labeled, df_unlabeled]).sort_index()

print(df.index)
print(df_result.index)

RangeIndex(start=0, stop=1000, step=1)
Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       990, 991, 992, 993, 994, 995, 996, 997, 998, 999],
      dtype='int64', length=1000)


Индексы ~совпадают

In [19]:
# Сохраню в xlsx, так как много строк имеют \n и \r символов, что ломает разделение строк в csv
df_result[['Id', 'text', 'answer']].to_excel('data/processed/result.xlsx', index=False)
df_result.to_excel('data/processed/result_detailed.xlsx', index=False)

## Label Propagation (не стал делать, так как того, что сверху хватило)

In [40]:
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.preprocessing import LabelEncoder